# Labolatorium 10
## Dyskretna transformacja Fouriera


In [ ]:
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
import time

---
### Zadanie 1 FFT

1. Zaimplementuj funkcję realizującją DFT jako iloczyn macierzy Fouriera $F_n$ i n-elementowego wektora wejściowego ($y = F_nx$).

$$\large
n = 2^r \\ \large
[F_n]_{jk} = E^{jk} \\ \large
E = e^{- \frac{2 \pi i}{n}} = cos(\frac{2\pi}{n}) - isin(\frac{2\pi}{n}) = \omega
$$

In [ ]:
def DFT(x):
    n = x.shape[0]
    Fn = np.array([[np.exp(2*np.pi*1j*i*k/n) for i in range(n)] for k in range(n)], dtype=np.complex_)
    return Fn @ x

W powyższej funkcji korzystamy z typu `np.complex_` z modułu `numpy` dla liczb zespolonych (przy tworzeniu macierzy Furiera `Fn`)

2. Zaimplementuj również IDFT korzystając z tożsamości:
$$\large
F^{-1}_ny = \frac{\overline{F_n}y}{n} = \frac{\overline{F_n\overline{y}}}{n}
$$

gdzie $\overline{x}$ to sprzężenie zespolone $x$.

Sprawdź poprawność działania funkcji realizującej DFT stosując transformację odwrotną (x = $F^{-1}_ny$) oraz porównując uzyskane wyniki z wyjściem funkcji bibliotecznej.

In [ ]:
def IDFT(y):
    n = y.shape[0]
    Fn = np.array([[np.exp(2*np.pi*1j*i*k/n) for i in range(n)] for k in range(n)], dtype=np.complex_)
    return (Fn @ y.conj()).conj() / n

`.conj()` zwraca nam sprzężenie zespolone macierzy numpy.

#### Sprawdzenie poprawności działania i porównanie z funkcjami bibliotecznymi

Funkcje bibilioteczne:
- `fft` z biblioteki `numpy` - transformacja Fouriera
- `ifft` z biblioteki `numpy` - odwrotna transformacja Fouriera

In [ ]:
def get_simple_signal(r):
    n = 2**r
    x = np.arange(n)
    return np.sin(2*np.pi*x/np.float64(n))

In [ ]:
signals = np.array([get_simple_signal(r) for r in [2, 4, 6]])

Do testów użyje sygnałów wygenerowanych ze wzoru $X_i = \sin(\frac{2\pi x_i}{n})$, gdzie $n = 2^r$, a $X = [x_0, x_1, ..., x_n]$, dla $r \in \{2, 4, 6\}$.

In [ ]:
def test_simple_dft(signal):
    y = DFT(signal)
    x = IDFT(y)
    y_lib = fft(signal)
    x_lib = ifft(y_lib)
    print(f"\nSignal of size {signal.shape[0]}")
    print(f"\ny =?= y_lib -> {np.allclose(abs(y), abs(y_lib))}")
    print(f"x =?= signal -> {np.allclose(x, signal)}")
    print(f"x =?= x_lib -> {np.allclose(x, x_lib)}")

In [ ]:
for signal in signals:
    test_simple_dft(signal)

Na podstawie powyższych wyników widzimy, że:
1. Nasza funkcja `DFT` poprawnie dokonuje transformacji Fouriera (jest równa wynikowi funkcji bibliotecznej z dokładnością do znaku).
2. Nasza funkcja `IDFT` poprawnie wykonuje odwrotną transformację fouriera (przepuszczając sygnał przez `DFT` a następnie `IDFT` otrzymujemy na wyjściu ten sam sygnał).
3. Wynik funkcji `IDFT` jest równy wynikowi funkcji bibliotecznej `ifft` co potwierdza poprawność jej działania.

---
#### 3.
Zaimplementuj rekurencyjny algorytm Cooleya-Turkeya realizujący szybką transformację Fouriera (FFT). Porównaj szybkość jego działania z implementacją biblioteczną z mnożeniem wektora przez macierz $F_n$ dla danych o różnych rozmiarze

In [ ]:
def FFT(x, N, s):

    if N == 1:
        return np.array(x[:1], dtype=np.complex_)
    
    n = N//2
    
    X = np.concatenate((FFT(x, n, 2*s), FFT(x[s:], n, 2*s)), axis=0)
    
    f_n = np.exp(-2j * np.pi * np.arange(n) / N)
    
    return np.concatenate((X[:n] + f_n * X[n:], X[:n] - f_n * X[n:]), axis=0)

def Cooley_Tukey(x):
    return FFT(x, x.shape[0], 1)

#### Sprawdzenie poprawności działania i porównanie z funkcjami bibliotecznymi (w tym pomiar czasów)

Tym razem dane testowe będziemy budowali w ten sam sposób ale dla $ r \in \{4, 8, 12\}$.

In [ ]:
signals = np.array([get_simple_signal(r) for r in [4, 8, 12]])

In [ ]:
def timeit(f, x):
    start = time.time()
    res = f(x)
    stop = time.time()
    return res, (stop - start)

def test_fft(signal):
    y_slow, y_slow_time = timeit(DFT, signal)
    y_fast, y_fast_time = timeit(Cooley_Tukey, signal)
    y_lib, y_lib_time = timeit(fft, signal)
    
    print(f"\nSignal of size {signal.shape[0]}")
    print(f"\nFFT y =?= DFT y -> {np.allclose(abs(y_fast), abs(y_slow))}")
    print(f"FFT y =?= lib y -> {np.allclose(abs(y_fast), abs(y_lib))}")
    print(f"DFT took {y_slow_time}s")
    print(f"FFT took {y_fast_time}s")
    print(f"lib fft took {y_lib_time}s")

In [ ]:
for signal in signals:
    test_fft(signal)

Widzimy, że szybka transformacja Fouriera `FFT` jest zdecydowanie szybsza od zwykłego algorytmu `DFT`. W dalszym ciągu jednak najszybsza pozostaje funkcja biblioteczna `numpy.fft.fft`.

---
### Zadanie 2 DFT w 1D

* Wygeneruj dwa sygnału czasowo-amplitudowe:

a) Sygnał będący sumą pięciu sygnałów sinusoidalnych o różnych częstotliwościach

In [ ]:
def get_signal_sin(cycles_num, frequency, n=256):
    N = cycles_num * n
    x = np.arange(N)
    return np.sin(2 * np.pi * frequency * x / float(n))

In [ ]:
def plot_input_signal(signal, n=256):
    N = signal.shape[0]
    X = 2 * np.pi * np.arange(N) / float(n)
    fig = plt.figure(figsize=(14,8))
    plt.plot(X, signal)

Poniżej możemy zobaczyć osobno pięć sygnałów sinusoidalnych, które będziemy wykorzystywać w dalszej części zadania

In [ ]:
sinuses = [get_signal_sin(4, freq) for freq in [1, 2, 4, 6, 9]]
for sinus in sinuses:
    plot_input_signal(sinus)

In [ ]:
sin_sum = np.sum(sinuses, axis=0)

Poniżej na wykresie przedstawiam sumę powyższych pięciu sygnałów sinusoidalnych.

In [ ]:
plot_input_signal(sin_sum)

b) Sygnał złożony z pięciu sygnałów o tych samych częstotliwościach co w punkcie a), ale ułożonych przedziałami, tzn. w każdym z pięciu przedziałów o tej samej szerokości występuje sygnał o jednej częstotliwości.

In [ ]:
sin_concat = np.concatenate(sinuses, axis=0)

Poniżej na wykresie przedstawiam konkatenację pięciu sygnałów sinusoidalnych.

In [ ]:
plot_input_signal(sin_concat)

* Dokonaj transformacji sygnałów a) i b) do domeny częstotliwościowej, porównaj otrzymane wyniki. Przedstaw na osobnych wykresach część rzeczywistą i część urojoną wyniku transformacji.

Ponieważ nasz algorytm działa tylko dla wyników $N = 2^r$, "obetniemy" sygnały tak, aby ich długości były potęgą dwójki:

In [ ]:
sin_sum.shape ## Tutaj jest ok

In [ ]:
sin_concat.shape ## Tu musimy uciąć

In [ ]:
sin_concat = sin_concat[:2**int(np.log2(sin_concat.shape[0]))]
sin_concat.shape

In [ ]:
plot_input_signal(sin_concat)

Teraz możemy wreszcie dokonać transformacji sygnałów a) i b) używając napisanego przez nas algorytmu `Cooleya-Tukeya`. Ponieważ wektor wynikowy jest symetryczny względem swojego środka, na wykresie pokażę tylko pierwszą jego połowę 

In [ ]:
sin_sum_fft = Cooley_Tukey(sin_sum)
sin_concat_fft = Cooley_Tukey(sin_concat)

In [ ]:
def plot_fft(Y, title="", n=256):
    N = Y.shape[0]
    X = np.arange(N//2) * n / float(N)
    fig, axes = plt.subplots(2, figsize=(14,8))
    fig.suptitle(title, fontsize=22)
    axes[0].plot(X, np.real(Y)[:N//2])
    axes[0].set_title("Czesc rzeczewista", fontsize=20)
    axes[1].plot(X, np.imag(Y)[:N//2])
    axes[1].set_title("Czesc urojona", fontsize=20)

In [ ]:
plot_fft(sin_sum_fft, title="Suma pieciu sygnalow sinusoidalnych")

In [ ]:
plot_fft(sin_concat_fft, title="Konkatenacja sygnalow sinusoidalnych")

#### Porównanie wykresów

Na pierwszym wykresie (suma sygnałów) możemy zaobserwować, że część rzeczywista przypomina nam sygnał radiowy bądź też inny sygnał o nieciągłym natężeniu. Część urojona tego wykresu jest praktycznie cały czas równa 0 za wyjątkiem tzw. "pików". W przypadku konkatenacji sygnałów, wykres wygląda nieco inaczej - Na początku mamy bardzo dużo różnych wartości zarówno w domenie rzeczywistej jak i urojonej, żeby następnie wartości praktycznie przez resztę przedziału były równe 0.